# Episode 5 - Loops
This notebook is based on a snapshot of [Episode 5](https://kmichali.github.io/SC-shell-novice/05-loop/index.html) of the [Unix Shell lesson](https://kmichali.github.io/SC-shell-novice/) from the [Software Carpentry](https://software-carpentry.org). The original material has more detail.

### Questions:
- How can I perform the same actions on many different files?

### Objectives:
- Write a loop that applies one or more commands separately to each file in a set of files.
- Trace the values taken on by a loop variable during execution of the loop.
- Explain the difference between a variable’s name and its value.
- Explain why spaces and some punctuation characters shouldn’t be used in file names.

<hr style="border: solid 1px red; margin-top: 1.5% ">

Loops are a programming construct which allow us to repeat a command or set of commands for each item in a list. As such they are key to productivity improvements through automation. Similar to wildcards, using loops also reduces the amount of typing required (and hence reduces the number of typing mistakes).

Suppose we have several hundred genome data files (e.g., basilisk.dat, minotaur.dat, and unicorn.dat). For this example, we’ll use the **`creatures`** directory which only has three example files, but the principles can be applied to many many more files at once.

The structure of these files is the same: the common name, classification, and updated date are presented on the first three lines, with DNA sequences on the following lines. Let’s look at the files:

In [8]:
cd data-shell/creatures

/Users/katerina/GS_comm_line/notebooks/data-shell/creatures


In [2]:
%%bash
head -n 5 *.dat

==> basilisk.dat <==
COMMON NAME: basilisk
CLASSIFICATION: basiliscus vulgaris
UPDATED: 1745-05-02
CCCCAACGAG
GAAACAGATC

==> minotaur.dat <==
COMMON NAME: minotaur
CLASSIFICATION: bos hominus
UPDATED: 1765-02-17
CCCGAAGGAC
CGACATCTCT

==> unicorn.dat <==
COMMON NAME: unicorn
CLASSIFICATION: equus monoceros
UPDATED: 1738-11-24
AGCCGGGTCG
CTTTACCTTA


## Print second line from every file
<hr style="border: solid 1px gray; margin-top: 1.5% ">

We would like to print out the classification for each species, which is given on the second line of each file. For each file, we would need to execute the command head -n 2 and pipe this to tail -n 1. 

One would be tempted to use wildcard.  However, this approach does not work in pipe, we only get the classification for the last file.

In [4]:
%%bash
# this does not work as intended
head -n 2 *.dat | tail -n 1

CLASSIFICATION: equus monoceros


We’ll use a loop to solve this problem, but first let’s look at the general form of a loop:

```
for thing in list_of_things
do
    operation_using $thing    # Indentation within the loop is not required, but aids legibility
done

```
Note that the loop construct takes four lines and it contains compulsory keywords - **`for`**, **`in`**, **`do`** and **`done`**. When the shell sees the keyword **`for`**, it knows to repeat a command (or commands) in the body of the loop (indented above) once for each item in a list. Each time the loop runs (called an iteration), an item in the list is assigned in sequence to the variable (called "thing" above), and the commands inside the loop are executed, before moving on to the next item in the list. 

Inside the loop, we call for the variable’s value by putting **`\$`** in front of it. The **`\$`** tells the shell interpreter to treat the variable as a variable name and substitute its value in its place, rather than treat it as text or an external command.

A loop can be written on a single line as well:

```
for thing in list_of_things; do operation_using $thing ; done

```

Using a loop, we can print out the second line from every **`.dat`** file.

In [5]:
%%bash
for filename in basilisk.dat minotaur.dat unicorn.dat
do
    head -n 2 $filename | tail -n 1
done

CLASSIFICATION: basiliscus vulgaris
CLASSIFICATION: bos hominus
CLASSIFICATION: equus monoceros


Loops also work with wildcards.

In [10]:
%%bash
for filename in *.dat
do 
   head -n 2 $filename | tail -n 1
done

CLASSIFICATION: basiliscus vulgaris
CLASSIFICATION: bos hominus
CLASSIFICATION: equus monoceros


The variable name can be changed but should be informative.

In [11]:
%%bash
for creature in *.dat
do 
   head -n 2 $creature | tail -n 1
done

CLASSIFICATION: basiliscus vulgaris
CLASSIFICATION: bos hominus
CLASSIFICATION: equus monoceros


One can follow the progress of a loop by printing out the value of the loop variable in each iteration with **`echo`**. 

In [12]:
%%bash
for filename in *.dat
do 
   echo $filename
   head -n 2 $filename | tail -n 1
done

basilisk.dat
CLASSIFICATION: basiliscus vulgaris
minotaur.dat
CLASSIFICATION: bos hominus
unicorn.dat
CLASSIFICATION: equus monoceros


Be very careful with loops that iterate through list of files names that have spaces, these may be interpreted as two separate files; e.g. **`purple unicorn.dat`** could be considered as two files named **`purple`** and **`unicorn.dat`**.  The problem can be avoided by surrounding the file name with quotes.

## Save a copy of every .dat file
<hr style="border: solid 1px gray; margin-top: 1.5% ">

The next task is to make a copy of every .dat file and save it under a new name.  For example, a copy of **`unicorn.dat`** will be saved as **`original-unicorn.dat`**.

Similarly to using head in an example above, we cannot resort to using the copy command with a wildcard (**`*.dat`**); **`cp *.dat original-*.dat`** will not work because the wildcard will be expanded first. The command that the shell will try to execute will look as follows **`cp basilisk.dat minotaur.dat unicorn.dat original-*.dat`**; the only way that **`copy`** could work with multiple arguments is if the last argument is a valid directory name (this would not accomplish our task though).



In [15]:
%%bash
cp *.dat original-*.dat

usage: cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file ... target_directory


We must use a loop for this task.  It is a good practice to use a "dry run" first. Instead of executing the body of the loop, one can print the command to make sure that the loop will do what was intended; for example **`cp $filename original-$filename`** will become **`echo "cp $filename original-$fi
lename"`**.

Note: The copy command uses the following construct **`original-$filename`** to add a plain text to a variable in order to create a new filename (e.g., original-unicorn.dat). Since the plain text goes in front of the variable name, it is safe.  The other way around **`$filename-original`** could be problematic, the shell may not be able to tell where the variable name ends.  In this case, we have to use curly braces to clearly delimit the variable name - **`${filename}-original`**.

In [14]:
%%bash
for filename in *.dat
do 
  echo "cp $filename original-$filename"
done

cp basilisk.dat original-basilisk.dat
cp minotaur.dat original-minotaur.dat
cp unicorn.dat original-unicorn.dat
